In [1]:
import os

# 更改缓存路径
os.environ["HF_HOME"] = "D:/huggingface"
os.environ["HF_DATASETS_CACHE"] = "D:/huggingface/datasets"

# tokenizer编码文本的处理过程

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203031553359.png)

- `tokenizer(sequence)`方法实现了**tokenization**，处理**special tokens**和转化为**input ids**三个过程。
- `tokenizer.tokenize(sequence)`来实现**tokenization**这个过程，输出是 list of strings, or tokens。
- `tokenizer.convert_tokens_to_ids(tokens)`来实现将**tokens转换为ids**。

注意：tokenizer(sequence)方法，可能会在开头添加[CLS]结尾添加[SEP]等特殊词。这是因为模型是用这些进行预训练的，所以为了获得相同的推理结果，我们还需要添加它们。注意有些ckeckpoint不加特殊词，或者加不同的词；模型也可以仅在开头或仅在结尾添加这些特殊词。在任何情况下，tokenizer()都会自动处理这些。。。而使用tokenizer.tokenize(sequence)则不会处理这些。

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
print(type(encoding))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


* 分词器的输出不是一个简单的 Python 字典；我们得到的实际上是一个特殊的 BatchEncoding 对象。它是字典的子类（这就是为什么我们之前能够毫无问题地索引到该结果），但它具有主要由快速分词器使用的附加方法。
* 除了它们的并行化能力之外，快速标记器的关键功能是它们始终跟踪最终标记来自的文本的原始跨度——我们称之为偏移映射的特性。这反过来又解锁了一些功能，例如将每个单词映射到它生成的标记或将原始文本的每个字符映射到它内部的标记，反之亦然。

# 检查是否是快速分词器

我们有两种方法来检查我们的分词器是快还是慢。我们可以检查tokenizer的is_fast属性。

In [3]:
tokenizer.is_fast

True

In [4]:
encoding.is_fast

True

# 查看分词的token

In [5]:
encoding

{'input_ids': [101, 1422, 1271, 1110, 156, 7777, 2497, 1394, 1105, 146, 1250, 1120, 20164, 10932, 10289, 1107, 6010, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
encoding.tokens()

['[CLS]',
 'My',
 'name',
 'is',
 'S',
 '##yl',
 '##va',
 '##in',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 'Face',
 'in',
 'Brooklyn',
 '.',
 '[SEP]']

## 使用word_ids()方法获取每个token来自的单词的索引

在这种情况下，索引 5 处的标记是##yl，它是原始句子中单词“Sylvain”的一部分。我们还可以使用word_ids()方法获取每个token来自的单词的索引：

In [7]:
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

**我们可以看到，tokenizer 的特殊标记[CLS]和[SEP]被映射到None，然后每个标记都被映射到它起源的单词。这对于确定标记是否位于单词的开头或两个标记是否在同一个单词中特别有用。我们可以依赖##前缀，但它只适用于类似 BERT 的分词器；此方法适用于任何类型的标记器，只要它是快速的。在下一章中，我们将看到如何使用此功能将每个单词的标签正确地应用于命名实体识别 (NER) 和词性 (POS) 标记等任务中的标记。我们还可以使用它来掩盖来自掩码语言建模（一种称为全词掩码的技术）中来自同一单词的所有标记。**

> 单词是什么的概念很复杂。例如，“I'll”（“I will”的缩写）算作一个词还是两个词？它实际上取决于分词器和它应用的预分词操作。一些分词器只是在空格上拆分，所以他们会将其视为一个词。其他人在空格顶部使用标点符号，因此将其视为两个词。

## 使用word_to_chars()将单词的索引转化为单词

In [8]:
# 将单词的索引id转化为单词字符串，实际返回的是单词字符串的起始和终止位置索引，然后对原始输入字符串进行切片，就可以看到单词
# the word_ids() method told us that ##yl is part of the word at index 3, but which word is it in the sentence? We can find out like this:
start, end = encoding.word_to_chars(3)
print(start, end)
example[start:end]

11 18


'Sylvain'

# Inside the token-classification pipeline

In [9]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.99590707,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932106,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

该模型正确地将“Sylvain”生成的每个token识别为一个人，将“Hugging Face”生成的每个token识别为一个组织，将“Brooklyn”生成的每个token识别为一个位置。

**我们还可以要求pipeline将对应于同一实体的标记组合在一起：**

In [10]:
from transformers import pipeline

token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

选取的aggregation_strategy将更改为每个分组实体计算的分数。"simple"分数只是给定实体中**每个标记的分数的平均值**：例如，“Sylvain”的分数是我们在前面示例中看到的标记、S、##yl、##va和##in的分数的平均值。其他可用的策略是：

* "first"，其中每个实体的分数是该实体的第一个令牌的分数（因此对于“Sylvain”，它将是 0.993828，令牌的分数S）
* "max"，其中每个实体的得分是该实体中标记的最大得分（因此对于“Hugging Face”，它将是 0.98879766，即“Face”的得分）
* "average"，其中每个实体的得分是组成该实体的**单词得分的平均值**（因此，对于“Sylvain”，与"simple"策略没有区别，但“Hugging Face”的得分为 0.9819，即“Hugging”的得分为 0.975，“Face”的得分为 0.98879）

In [11]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

In [12]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 19])
torch.Size([1, 19, 9])


**我们有一个包含 19 个标记的 1 个序列的批次，模型有 9 个不同的标签，因此模型的输出具有 1 x 19 x 9 的形状。与文本分类管道一样，我们使用 softmax 函数来转换这些 logits到概率，我们使用 argmax 来获得预测（注意我们可以在 logits 上使用 argmax，因为 softmax 不会改变顺序）：**

In [13]:
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(inputs.tokens())
print(len(inputs.tokens()))
print(predictions)
print(len(predictions))

['[CLS]', 'My', 'name', 'is', 'S', '##yl', '##va', '##in', 'and', 'I', 'work', 'at', 'Hu', '##gging', 'Face', 'in', 'Brooklyn', '.', '[SEP]']
19
[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]
19


![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203041038648.png)

**model.config.id2label属性包含索引到标签的映射，我们可以使用这些映射来理解预测：**

In [14]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

正如我们之前看到的，有 9 个标签：O是不在任何命名实体中的标记的标签（它代表“外部”），然后我们为每种类型的实体（杂项、人员、组织、和位置）。标签B-XXX指示令牌位于实体的开头，XXX标签I-XXX指示令牌位于实体内部XXX。例如，在当前示例中，我们希望我们的模型将标记分类S为B-PER（个人实体的开头）和标记##yl，##va以及##in（I-PER个人实体内部）。

在这种情况下，您可能会认为该模型是错误的，因为它为所有这四个令牌提供了标签I-PER，但这并不完全正确。这些 B 和 I 标签实际上有两种格式：IOB1 和 IOB2。IOB2 格式（下面的粉红色）是我们介绍的格式，而在 IOB1 格式（蓝色）中，以 B- 开头的标签仅用于分隔同一类型的两个相邻实体。我们使用的模型是在使用该格式的数据集上进行微调的，这就是它将标签 I-PER 分配给 S 令牌的原因。

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203040958489.png)

In [15]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S'}, {'entity': 'I-PER', 'score': 0.9981549382209778, 'word': '##yl'}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in'}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9887976050376892, 'word': 'Face'}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn'}]


管道还为我们提供了有关原始句子中每个token的开始和结束的信息。这就是我们的偏移映射将发挥作用的地方。要获得偏移量，我们只需要在将分词器应用于输入时设置return_offsets_mapping=True：

In [16]:
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 12),
 (12, 14),
 (14, 16),
 (16, 18),
 (19, 22),
 (23, 24),
 (25, 29),
 (30, 32),
 (33, 35),
 (35, 40),
 (41, 45),
 (46, 48),
 (49, 57),
 (57, 58),
 (0, 0)]

每个元组是对应于每个标记的文本范围，其中 （0， 0） 是为特殊标记保留的。我们之前看到索引5处的令牌是##yl，此处有（12，14）作为偏移量。如果我们在示例中抓取相应的切片：

In [17]:
example[12:14]

'yl'

In [18]:
results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.9981549382209778, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887976050376892, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


# 对实体进行分组

使用偏移量来确定每个实体的开始键和结束键非常方便，但该信息并不是绝对必要的。但是，当我们想要将实体组合在一起时，偏移量将为我们节省大量混乱的代码。例如，如果我们想将令牌 Hu、##gging 和 Face 组合在一起，我们可以制定特殊规则，规定在删除 ## 时应附加前两个令牌，并且 Face 应添加空格，因为它不以 ## 开头 — 但这仅适用于此特定类型的分词器。我们必须为 SentencePiece 或 Byte-Pair-Encoding 赋能器编写另一组规则（在本章后面讨论）。

有了偏移量，所有自定义代码都会消失：我们只需要获取原始文本中以第一个标记开头并以最后一个标记结尾的跨度。因此，对于令牌 Hu、##gging 和 Face，我们应该从字符 33（Hu 的开头）开始，在字符 45（Face 的结尾）之前结束：

In [19]:
example[33:45]

'Hugging Face'

要编写在对实体进行分组时对预测进行后处理的代码，我们将对连续的实体进行分组并标记为 I-XXX，但第一个实体除外，它可以标记为 B-XXX 或 I-XXX（因此，当我们获得 O、新类型的实体或告诉我们正在启动相同类型的实体的 B-XXX 时，我们将停止对实体进行分组）：

In [20]:
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)

[{'entity_group': 'PER', 'score': 0.9981694370508194, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796018997828165, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


在QA任务中，这些偏移量非常有用的。